In [2]:
import os

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [4]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True  
session = tf.Session(config=config)
 
KTF.set_session(session)

Using TensorFlow backend.


In [5]:
import numpy as np
import keras_bert

In [6]:
from hypers import *
from preprocess import *
from Knowledge import *
from Models import *

# Load BERT Model

In [7]:
bert_path='/data1/qitao/BERT/chinese_L-12_H-768_A-12/'
emb_root_path = '/data/data/qit/cnchar.model'

In [ ]:
bert_word_index_path=bert_path+'vocab.txt'

bert_config_path = os.path.join(bert_path,'bert_config.json')
bert_checkpoint_path = os.path.join(bert_path,'bert_model.ckpt')
bert_vocab_path = os.path.join(bert_path,'vocab.txt')

bert_voca = read_bert_word_index(bert_word_index_path)

bert = keras_bert.load_trained_model_from_checkpoint(bert_config_path,
                                                        bert_checkpoint_path, 
                                                        training=False,
                                                        trainable=True,
                                                        seq_len=max_text_length,
                                                        )

# Load and Preprocess Data

In [ ]:
TrainData,TestData,Centorid = read_data()

In [ ]:
char_dict = {}
tp_dict = {}
    
train_input, train_label, train_mask = parse_datav2(TrainData,char_dict,tp_dict,bert_voca)
test_input, test_label, test_mask = parse_datav2(TestData,char_dict,tp_dict,bert_voca)
train_mask = train_mask.reshape((train_mask.shape[0],max_entity_num*max_entity_num,))
train_label = train_label.reshape((train_label.shape[0],max_entity_num*max_entity_num,1))

In [ ]:
emb_matrix=Init_char_embedding(emb_root_path,char_dict)

# Model Training

In [ ]:
use_bert = True
char_cnn = True
char_lstm = True
entity_type = True
entity_cnn = True
entity_lstm = True

model = get_modelv2(emb_matrix,bert,tp_dict,use_bert,char_cnn,char_lstm,entity_type,entity_cnn,entity_lstm)
model.compile(Adam(0.0001),loss='binary_crossentropy',sample_weight_mode="temporal")

for i in range(25):
    model.fit(train_input,train_label,epochs = 5,batch_size=2,sample_weight=train_mask)
    pred = infer(model,test_input,test_mask)
    g = evaluation(pred,test_label,test_mask)
    print('test',g)

Epoch 1/5
242/242 [==============================] - 343s 1s/step - loss: 0.1331
Epoch 2/5
242/242 [==============================] - 323s 1s/step - loss: 0.0654
Epoch 3/5
242/242 [==============================] - 318s 1s/step - loss: 0.0340
Epoch 4/5
242/242 [==============================] - 317s 1s/step - loss: 0.0172
Epoch 5/5
242/242 [==============================] - 315s 1s/step - loss: 0.0101
test (0.9887448294675105, 0.6330081000890088, 0.9576660630890312, 0.762205597033837)
Epoch 1/5
242/242 [==============================] - 321s 1s/step - loss: 0.0069
Epoch 2/5
242/242 [==============================] - 317s 1s/step - loss: 0.0044
Epoch 3/5
242/242 [==============================] - 315s 1s/step - loss: 0.0047
Epoch 4/5
242/242 [==============================] - 312s 1s/step - loss: 0.0035
Epoch 5/5
242/242 [==============================] - 304s 1s/step - loss: 0.0023
test (0.9910136997799327, 0.8111192930033947, 0.8651089730154026, 0.8372446555023993)
Epoch 1/5
242/242 [